<a href="https://colab.research.google.com/github/Aravindmodala/Aravindmodala/blob/main/SENTIMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy  as np

df = pd.read_csv(r"/content/test(in).csv")

In [11]:
df

,Subject,body,date,from
0,EnronOptions Update!,"EnronOptions Announcement\n\n\nWe have updated the EnronOptions =01) Your Stock Option Program web site! =\nThe=20\nweb site now contains specific details of the EnronOptions Program includin=\ng=20\nthe December 29, 2000 grant price and additional information on employee=20\neligibility.\n\nAccess EnronOptions =01) Your Stock Option Program web site by clicking on=\n=20\nhttp://hrweb.enron.com or by typing the address into your web browser.\n\nWe hope you find the revised site helpful in addressing your questions abou=\nt=20\nthe EnronOptions Program. If you have any additional questions, please=20\ncontact your H.R. Representative.\n\nStay tuned for more information throughout March and April 2001.",5/10/2010,sally.beck@enron.com
1,(No Subject),"Marc,\n\nUnfortunately, today is not going to work for the revenue model for mid & \nback office services meeting. How about this coming Monday, Feb 26 from 1:00 \n- 2:30 pm? I have penciled in that time for Sally, Mary Solmonson & James \nScribner. Please let me know if that will work for you.\n\nThanks,\nPatti x39106",7/29/2010,eric.bass@enron.com
2,Phone Screen Interview - Shannon L. Burnham,"When: Wednesday, June 06, 2001 10:00 AM-11:00 AM (GMT-06:00) Central Time (US & Canada).\nWhere: @ 10:00am CST (225) 931-8111\n\n*~*~*~*~*~*~*~*~*~*\n\n",7/25/2011,sally.beck@enron.com
3,RE: My new work email,we were thinking papasitos (we can meet somewhere closer to you this time) \nagain at around 8\n\n\n\n\nJason Bass <JBass@stdauto.com> on 02/21/2001 11:22:45 AM\n,3/25/2010,johnny.palmer@enron.com
4,Bet,"Since you never gave me the $20 for the last time we bet on CU/Texas, I figured I would give you the chance to go double or nothing on the second round. I think the line is Texas -9. Let me know.",5/21/2011,lydia.delgado@enron.com
...,...,...,...,...
2186,Re: Resume,Thanks for the resume. She has had some good experience. Do you know much \nabout her performance here at Enron? Can I call and talk with her directly? \nI would certainly keep the conversation confidential at this point. --Sally \n\n\n\n\nVince J Kaminski\n11/21/2000 08:14 AM\n,6/17/2011,johnny.palmer@enron.com
2187,"Final Schedule - Wednesday, May 2, 2001 - Jesus O. Guerra","Attached please find the following documents:\n\n\n \n\n\nThanks,\nLydia\n\nx3-9338",1/20/2011,johnny.palmer@enron.com
2188,(No Subject),"Good to finally hear from. Judging from your e-mail, it sounds like you had \nsome fun in your exercises- I guess that English degree is finally paying off.\nLuckily you missed out on another series of disappointments from the Longhorn \nfootball team. It seems as though they are improving, however. Hopefully we \nwill be playing for the National Championship next year. \n\nHow are things otherwise? When are you coming into town again?\n\nLater-\n\nEric",1/2/2011,don.baughman@enron.com
2189,League is Set,"It looks like we have our 12 teams. We will probably keep the same rules \nfrom last year unless anybody has an objection. Still to be determined are \nleague fees, draft date , and free agent pickup protocol. I suggest that we \ndraft on August 27th at the Enron building on floor 26. For those of you \nthat can't make it, you can draft over the internet. We will be using cbs \nsportsline again - I think that will be $8/person.\n\nLet me know if anyone has a problem with the draft date, i.e. can't make it, \ntoo early, too late.\nIf not, THE MEXIMULLETS look forward to a return to Prominence this year. \nYou all do not stand a chance!\n\n-Eric",3/11/2011,rhonda.denton@enron.com


In [12]:
df['Subject'] = df['Subject'].replace('(No Subject)', ' ')

In [13]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
import re

def clean_subject(subject):
    subject = str(subject)

    # Remove RE/FW/FWD/etc
    subject = re.sub(r'\b(RE|FW|FWD|From|To|CC|Subject):\s*', '', subject, flags=re.I)

    # Remove email addresses
    subject = re.sub(r'\S+@\S+', '', subject)

    # Remove angle brackets or [] around names
    subject = re.sub(r'[<>[\](){}]', '', subject)

    # Remove excessive whitespace
    subject = re.sub(r'\s+', ' ', subject).strip()

    return subject
df['Subject'] = df['Subject'].apply(clean_subject)

In [15]:
import re

def clean_body(text):
    text = str(text)

    # === Fix quoted-printable artifacts ===
    text = re.sub(r'=\n', '', text)
    text = re.sub(r'=\s+', '', text)
    text = re.sub(r'(?<=\w)=+(?=\w)', '', text)
    text = re.sub(r'=\d{2}', ' ', text)
    text = re.sub(r'(?<=\D)20(?=\D)', ' ', text)  # extra 20s

    # === Remove emails, URLs, and IDs ===
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'http\S+|www\S+', '', text)

    # === Remove numbers after CST/EST/phone pattern ===
    text = re.sub(r'\b(CST|EST|PST)?\s*\d{3}[\s-]?\d{3}[\s-]?\d{3,6}\b', '', text, flags=re.I)

    # === Remove extensions like x3-9338 ===
    text = re.sub(r'\bx\d{1,4}-?\d{2,6}\b', '', text)
    text = re.sub(r'(ext|extension)[\s:.]*\d{3,6}', '', text, flags=re.I)

    # === Remove symbols and bullets ===
    text = re.sub(r'\[image.*?\]', '', text, flags=re.I)
    text = re.sub(r'[<>\[\]{}()~@=*•|]+', '', text)

    # === Signature phrases and names ===
    text = re.sub(r'(Regards|Sincerely|Thanks|Best)[^\n]*', '', text, flags=re.I)
    text = re.sub(r'--\s*[A-Z][a-z]+\b.*', '', text)

    # === Scheduling metadata ===
    text = re.sub(r'(?i)(when|where|date|time):\s*[^\n\r]{0,80}', '', text)
    text = re.sub(r'\b\d{1,2}/\d{1,2}/\d{2,4}\b', '', text)
    text = re.sub(r'\b\d{4}-\d{2}-\d{2}\b', '', text)
    text = re.sub(r'\b(?:Mon|Tue|Wed|Thu|Fri|Sat|Sun)\b', '', text, flags=re.I)
    text = re.sub(r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\b', '', text, flags=re.I)
    text = re.sub(r'\b\d{1,2}:\d{2}(:\d{2})?\s*(AM|PM|am|pm)?\b', '', text)

    # === Normalize whitespace ===
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply
df['body'] = df['body'].apply(clean_body)


In [16]:
pd.set_option('display.max_colwidth', None)
df

,Subject,body,date,from
0,EnronOptions Update!,"EnronOptions Announcement We have updated the EnronOptions Your Stock Option Program web site! The web site now contains specific details of the EnronOptions Program including the 29, 2000 grant price and additional information on employee eligibility. Access EnronOptions Your Stock Option Program web site by clicking on or by typing the address into your web browser. We hope you find the revised site helpful in addressing your questions about the EnronOptions Program. If you have any additional questions, please contact your H.R. Representative. Stay tuned for more information throughout and 2001.",5/10/2010,sally.beck@enron.com
1,,", Unfortunately, today is not going to work for the revenue model for mid & back office services meeting. How about this coming Monday, 26 from - ? I have penciled in that time for Sally, Solmonson & James Scribner. Please let me know if that will work for you. Patti",7/29/2010,eric.bass@enron.com
2,Phone Screen Interview - Shannon L. Burnham,,7/25/2011,sally.beck@enron.com
3,My new work email,we were thinking papasitos we can meet somewhere closer to you this time again at around 8 Jason Bass on,3/25/2010,johnny.palmer@enron.com
4,Bet,"Since you never gave me the $ for the last time we bet on CU/Texas, I figured I would give you the chance to go double or nothing on the second round. I think the line is Texas -9. Let me know.",5/21/2011,lydia.delgado@enron.com
...,...,...,...,...
2186,Resume,about her performance here at Enron? Can I call and talk with her directly? I would certainly keep the conversation confidential at this point. Vince J Kaminski,6/17/2011,johnny.palmer@enron.com
2187,"Final Schedule - Wednesday, May 2, 2001 - Jesus O. Guerra",Attached please find the following documents: Lydia,1/20/2011,johnny.palmer@enron.com
2188,,"Good to finally hear from. Judging from your e-mail, it sounds like you had some fun in your exercises- I guess that English degree is finally paying off. Luckily you missed out on another series of disappointments from the Longhorn football team. It seems as though they are improving, however. Hopefully we will be playing for the National Championship next year. How are things otherwise? When are you coming into town again? Later- Eric",1/2/2011,don.baughman@enron.com
2189,League is Set,"It looks like we have our 12 teams. We will probably keep the same rules from last year unless anybody has an objection. Still to be determined are league fees, draft date , and free agent pickup protocol. I suggest that we draft on 27th at the Enron building on floor 26. For those of you that can't make it, you can draft over the internet. We will be using cbs sportsline again - I think that will be $8/person. Let me know if anyone has a problem with the draft date, i.e. can't make it, too early, too late. If not, THE MEXIMULLETS look forward to a return to Prominence this year. You all do not stand a chance! -Eric",3/11/2011,rhonda.denton@enron.com


In [17]:
df['full_text'] = df['Subject'] + ' ' + df['body']

In [ ]:
from transformers import pipeline

pipe = pipeline(model="facebook/bart-large-mnli")
results = pipe(df["full_text"].tolist(),
     candidate_labels=["positive", "negative", "neutral"],
     batch_size=8,
     cuda_device=0,
     truncation=True,
     max_length=512
)

Device set to use cpu


In [ ]:
print(results)

In [ ]:
df["raw_sentiment"] = [r["labels"][0].upper() for r in results]
df["raw_score"] = [r["scores"][0] for r in results]

In [ ]:
def zero_shot_labeling(result, threshold=0.55, margin=0.1):
    scores = result["scores"]
    labels = result["labels"]

    if scores[0] < threshold or (scores[0] - scores[1]) < margin:
        return "NEUTRAL"

    return labels[0].upper()

In [ ]:
df["smart_sentiment"] = [zero_shot_labeling(r) for r in results]

In [ ]:
df = df.drop(columns=["full_text"])

In [ ]:
df

In [ ]:
df = df.drop(columns=["raw_sentiment", "raw_score"])

# By below histplot we can see that there are much more positive, neutral emails than negative

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='smart_sentiment', palette='coolwarm')
plt.title("Sentiment Distribution")
plt.xlabel("Sentiment")
plt.ylabel("Count")
plt.show()


In [ ]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [ ]:
df['month'] = df['date'].dt.to_period('M')

In [ ]:
# Step 1: Group by month and sentiment, then unstack for plotting
monthly_counts = df.groupby(['month', 'smart_sentiment']).size().unstack(fill_value=0)

# Step 2: Plot the trend
monthly_counts.plot(kind='line', figsize=(12, 6), marker='o')

plt.title("Monthly Sentiment Trends")
plt.xlabel("Month")
plt.ylabel("Number of Messages")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.legend(title="Sentiment")
plt.show()

In [ ]:
df

### 📊 Sentiment Score Aggregation and Accuracy Justification

To ensure accurate analysis, each employee message was assigned a sentiment score based on its predicted label:

- `+1` for Positive
- `–1` for Negative
- `0` for Neutral

These scores were then aggregated on a monthly basis

 using the `employee` and `month` as grouping keys. The `month` was extracted using `.dt.to_period("M")`, ensuring that scores reset at the start of each new month. Aggregation was performed using:



In [ ]:
score_map = {"POSITIVE": 1, "NEGATIVE": -1, "NEUTRAL": 0}
df["score"] = df["smart_sentiment"].map(score_map)

In [ ]:
df['employee'] = df['from'].str.extract(r'^([^@]+)')
df['employee'] = df['employee'].str.replace('.', ' ').str.title()


In [ ]:
monthly_sentiment = df.groupby(['month', 'smart_sentiment']).size().unstack().fillna(0)

monthly_sentiment.plot(kind='area', stacked=True, figsize=(12,6), colormap='coolwarm')
plt.title("Monthly Distribution of Sentiment Types")
plt.ylabel("Message Count")
plt.xlabel("Month")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Step 1: Group and pivot the data
monthly_scores = df.groupby(["employee", "month"])["score"].sum().reset_index()
pivot_df = monthly_scores.pivot(index="employee", columns="month", values="score").fillna(0)

# Step 2: Plot the heatmap
plt.figure(figsize=(16, 10))
sns.heatmap(pivot_df, cmap="coolwarm", center=0, annot=True, fmt=".0f", linewidths=0.5)

plt.title("📊 Monthly Sentiment Score per Employee", fontsize=16)
plt.xlabel("Month")
plt.ylabel("Employee")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
monthly_scores = df.groupby(['employee', 'month'])['score'].sum().reset_index()
print(monthly_scores)

### ⚠️ Why Can an Employee Be Both Top Positive and FLR?

In some cases, an employee may appear in the **Top Positive** list and also be flagged as **Flight Risk (FLR)** in the same month. This happens because the two systems use different criteria:

- **Top Positive Employees** are determined based on their **total sentiment score** in a given month:
  1. Positive = +1
  2. Negative = -1
  3. Neutral = 0
  4. The top 3 with the highest total scores are selected.

- **Flight Risk (FLR)** is triggered **only** if an employee sends **4 or more negative messages** in a month, regardless of their overall score.

➡️ This means an employee could have a **high overall score** from many positive messages, but still be flagged FLR due to **frequent negativity**.

> Example:  
> 1. 6 Positive messages (+6)  
> 2. 4 Negative messages (–4)  
> 3. Net score = +2 (high → Top Positive)  
> 4. But 4 negatives → FLR triggered

This dual tagging highlights employees who may appear outwardly positive but are still showing frequent signs of frustration or disengagement — a valuable insight for deeper HR review.


### 🗓️ Monthly Top Sentiment Employees Summary

The table below displays the **Top 3 Positive** and **Top 3 Negative** employees for each month, based on their aggregated sentiment scores.

1. **Top Positive**: Employees with the highest total sentiment scores in that month.
2. **Top Negative**: Employees with the lowest (most negative) sentiment scores in that month.

This summary helps quickly identify standout performers as well as individuals who may require additional support or attention.


In [ ]:
summary = []
# Step 2: Loop through each unique month and get top pos and top neg
for m in monthly_scores['month'].unique():
    month_data = monthly_scores[monthly_scores['month'] == m]

    top_pos = month_data.sort_values(by=['score', 'employee'], ascending=[False, True]).head(3)
    top_pos_names = ', '.join(top_pos['employee'])

    top_neg = month_data.sort_values(by=['score', 'employee'], ascending=[True, True]).head(3)
    top_neg_names = ', '.join(top_neg['employee'])
    summary.append({
        'month': str(m),
        'Top Positive': top_pos_names,
        'Top Negative': top_neg_names
    })

# Step 3: Create and display summary table
summary_df = pd.DataFrame(summary)
summary_df = summary_df.sort_values(by='month').reset_index(drop=True)
display(summary_df)


###  Flight Risk Summary Explanation

The table below lists all employees who were flagged as **Flight Risk (FLR)** for specific months.

> **Definition of Flight Risk:**  
> An employee is flagged as FLR if they sent **4 or more negative messages in a single month**.

Each row in the table shows:
1. The **employee name**
2. The **month** in which they were flagged

If the same month appears multiple times, it means **multiple employees** were identified as flight risks in that month — this is expected and correct.

This summary helps HR and analysts identify patterns of disengagement or communication concerns that may require intervention.


In [ ]:
# Step 1: Filter only negative messages
negatives = df[df['smart_sentiment'] == 'NEGATIVE'].copy()

# Step 2: Count number of negative messages per employee per month
neg_count = negatives.groupby(['employee', 'month']).size().reset_index(name='neg_count')

# Step 3: Flag only those with 4 or more negative messages
flight_risks = neg_count[neg_count['neg_count'] >= 4]
flight_risk_pairs = set(zip(flight_risks['employee'], flight_risks['month']))

# Step 4: Define function to tag FLR in original df
def tag_flight_risk(row):
    return "FLR" if (row['employee'], row['month']) in flight_risk_pairs else ""

# Step 5: Apply it to the main DataFrame
df['risk_identification'] = df.apply(tag_flight_risk, axis=1)


In [ ]:
# Show all employees with at least one FLR tag
flight_risk_summary = df[df['risk_identification'] == 'FLR'][['employee', 'month']].drop_duplicates()

# Display
print("🔎 Employees flagged as Flight Risk (FLR):")
display(flight_risk_summary.sort_values(by=['month', 'employee']))


###  Employees Most Frequently Flagged as Flight Risk (FLR)

The bar chart above highlights the employees who were most frequently flagged as **Flight Risk (FLR)** over the dataset’s time period.

####  What this shows:
- An FLR flag indicates the employee sent **4 or more negative messages** in a single month.
1. The **y-axis** shows the **number of months** an employee was flagged.
2. The **x-axis** lists the employee names.

####  Key Insights:
1. **Lydia Delgado** stands out as the most frequently flagged employee, potentially indicating ongoing communication issues or morale concerns.
2. **Johnny Palmer** and **Patti Thompson** also show repeated months of negative sentiment, warranting further attention.

In [ ]:
flr_counts = df[df['risk_identification'] == 'FLR']['employee'].value_counts()

plt.figure(figsize=(10, 5))
flr_counts.plot(kind='bar', color='crimson')
plt.title("Employees Most Frequently Flagged as FLR")
plt.ylabel("Number of Months Flagged")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Developing a Linear regression model

In [ ]:
df

In [ ]:
df['word_count'] = df['body'].str.split().apply(len)

In [ ]:
df['word_count']

In [ ]:
df

In [ ]:
sns.lmplot(x='word_count', y='score', data=df, height=5, aspect=1.5)
plt.title("Linear Relationship: Word Count vs Sentiment Score")
plt.xlabel("Word Count")
plt.ylabel("Sentiment Score")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

features = ['word_count', ]
target = 'score'

X = df[features]
y = df[target]

